# Stack Semantics in Trax

- How to used `Select` and `Residual` which operates on elements in the stack
- Stack is a LIFO data structures

In [1]:
import numpy as np
from trax import layers as tl
from trax import shapes
from trax import fastmath

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


## The tl.Serial Combinator is Stack Oriented
**(3 + 4)*15 + 3** Perform this operation as follows

### Addition

In [4]:
def Addition():
    layer_name = "Addition" # Dont forget to give your  custom layer a name to identify
    
    # Custom function for the custom layer
    def func(x, y):
        return x + y
    
    return tl.Fn(layer_name, func)

# Test
add = Addition()
print(f'Layer Name: {add.name}, Expected Inputs: {add.n_in}, Promised Outputs: {add.n_out}')

# Inputs
x = np.array([3])
y = np.array([4])
print("-- Inputs --")
print("x :", x, "\n")
print("y :", y, "\n")

# Outputs
z = add((x, y))
print("-- Outputs --")
print("z :", z)

Layer Name: Addition, Expected Inputs: 2, Promised Outputs: 1
-- Inputs --
x : [3] 

y : [4] 

-- Outputs --
z : [7]


### Multiply

In [7]:
def Multiplication():
    layer_name = "Multiplication" # Dont forget to give your  custom layer a name to identify
    
    # Custom function for the custom layer
    def func(x, y):
        return x * y
    
    return tl.Fn(layer_name, func)

# Test
mul = Multiplication()
print(f'Layer Name: {mul.name}, Expected Inputs: {mul.n_in}, Promised Outputs: {mul.n_out}')

# Inputs
x = np.array([3])
y = np.array([4])
print("-- Inputs --")
print("x :", x, "\n")
print("y :", y, "\n")

# Outputs
z = add((x, y))
print("-- Outputs --")
print("z :", z)

Layer Name: Multiplication, Expected Inputs: 2, Promised Outputs: 1
-- Inputs --
x : [3] 

y : [4] 

-- Outputs --
z : [7]


### Implement using a Serial Combinator: **(3 + 4)*15 + 3**

In [9]:
# Serial Combinator
serial = tl.Serial(
    Addition(),
    Multiplication(),
    Addition()
)

# Initialization
x = (np.array([3]), np.array([4]), np.array([15]), np.array([3]))

serial.init(shapes.signature(x)) # initializing serial instance

print('Serial Modle')
print(serial, '\n')
print(f'Name: {serial.name}, Sublayers: {serial.sublayers}, Expected Inputs: {serial.n_in}, Promised Outputs: {serial.n_out}')

# Inputs
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = serial(x)
print("-- Outputs --")
print("y :", y)

Serial Modle
Serial_in4[
  Addition_in2
  Multiplication_in2
  Addition_in2
] 

Name: Serial, Sublayers: [Addition_in2, Multiplication_in2, Addition_in2], Expected Inputs: 4, Promised Outputs: 1
-- Inputs --
x : (array([3]), array([4]), array([15]), array([3])) 

-- Outputs --
y : [108]


### (3 + 4) * 3 + 4 - Using tl.Select
1. `4`
2. `3`
3. `tl.Select([0, 1, 0, 1])`
4. `add`
5. `mul`
6. `add`

- `tl.Select` requires a list of tuple 0-based indices to select elements relative to the top of the stak
- Top of the stack is `3` at index `0` and `4` at index `1`
- Select to add in an ordered manner to the top of the stack, after the command is `3` `4` `3` `4`
- The steps of the calculation for our example are shown below

1. Push(4) `Stack Tail` $\Rightarrow$  `4` $\Leftarrow$ `Stack Head`
2. Push(3) `Stack Tail` $\Rightarrow$ `4` `3` $\Leftarrow$ `Stack Head`
3. Push(select([0,1,0,1])) `Stack Tail` $\Rightarrow$ `4` `3` `4` `3` $\Leftarrow$ `Stack Head`
4. Push(Add Pop() Pop()) `Stack Tail` $\Rightarrow$ `4` `3` `7`$\Leftarrow$ `Stack Head`
5. Push(Mul Pop() Pop()) `Stack Tail` $\Rightarrow$ `4` `21`$\Leftarrow$ `Stack Head`
6. Push(Add Pop() Pop()) `Stack Tail` $\Rightarrow$ `25`$\Leftarrow$ `Stack Head`

In [11]:
serial = tl.Serial(
    tl.Select([0, 1, 0, 1]),
    Addition(),
    Multiplication(),
    Addition()
)

# Initialization
x = (np.array([3]), np.array([4]))
serial.init(shapes.signature(x))

print('Serial Modle')
print(serial, '\n')
print(f'Name: {serial.name}, Sublayers: {serial.sublayers}, Expected Inputs: {serial.n_in}, Promised Outputs: {serial.n_out}')

# Inputs
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = serial(x)
print("-- Outputs --")
print("y :", y)

Serial Modle
Serial_in2[
  Select[0,1,0,1]_in2_out4
  Addition_in2
  Multiplication_in2
  Addition_in2
] 

Name: Serial, Sublayers: [Select[0,1,0,1]_in2_out4, Addition_in2, Multiplication_in2, Addition_in2], Expected Inputs: 2, Promised Outputs: 1
-- Inputs --
x : (array([3]), array([4])) 

-- Outputs --
y : [25]


### (3 + 4) * 3 + 4 - Using tl.Select
1. Push(4) `Stack Tail` $\Rightarrow$  `4` $\Leftarrow$ `Stack Head`
2. Push(3) `Stack Tail` $\Rightarrow$ `4` `3` $\Leftarrow$ `Stack Head`
3. Push(select([0,1,0,1])) `Stack Tail` $\Rightarrow$ `4` `3` `4` `3` $\Leftarrow$ `Stack Head`
4. Push(Add Pop() Pop()) `Stack Tail` $\Rightarrow$ `4` `3` `7`$\Leftarrow$ `Stack Head`
5. Push(select([0], n_in=2)) `Stack Tail` $\Rightarrow$ `4` `7`$\Leftarrow$ `Stack Head`
6. Push(Mul Pop() Pop()) `Stack Tail` $\Rightarrow$ `28`$\Leftarrow$ `Stack Head`

In [13]:
serial = tl.Serial(
    tl.Select([0, 1, 0, 1]),
    Addition(),
    tl.Select([0], n_in=2),
    Multiplication()
)

# Initialization
x = (np.array([3]), np.array([4]))
serial.init(shapes.signature(x))

print('Serial Modle')
print(serial, '\n')
print(f'Name: {serial.name}, Sublayers: {serial.sublayers}, Expected Inputs: {serial.n_in}, Promised Outputs: {serial.n_out}')

# Inputs
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = serial(x)
print("-- Outputs --")
print("y :", y)

Serial Modle
Serial_in2[
  Select[0,1,0,1]_in2_out4
  Addition_in2
  Select[0]_in2
  Multiplication_in2
] 

Name: Serial, Sublayers: [Select[0,1,0,1]_in2_out4, Addition_in2, Select[0]_in2, Multiplication_in2], Expected Inputs: 2, Promised Outputs: 1
-- Inputs --
x : (array([3]), array([4])) 

-- Outputs --
y : [28]


## Residual Combinator 
- Residual networks are frequently used to make deep models easier to train
- Resuidual layer computes the element wise sum of the stack-top input with the output of the layer
- For ex. if we wanted the cumulatirve sum of the follwing series of computation `(3+4)*3 + 4`, result can be obtained with the use of the Residual combinator

1. 4
2. 3
3. tl.Select([0,1,0,1])
4. add
5. mul
6. tl.Residual

a. Push(4) `Stack Tail` $\Rightarrow$  `4` $\Leftarrow$ `Stack Head` $\Rightarrow$ `4`  
b. Push(3) `Stack Tail` $\Rightarrow$ `4` `3` $\Leftarrow$ `Stack Head` `11`  
c. Push(select([0,1,0,1])) `Stack Tail` $\Rightarrow$ `4` `3` `4` `3` $\Leftarrow$ `Stack Head` `21`  
d. Push(Add Pop() Pop()) `Stack Tail` $\Rightarrow$ `4` `3` `7`$\Leftarrow$ `Stack Head` `28`  
e. Push(Mul Pop() Pop()) `Stack Tail` $\Rightarrow$ `4` `21`$\Leftarrow$ `Stack Head` `39`  
f. Push(Add Pop() Pop()) `Stack Tail` $\Rightarrow$ `25`$\Leftarrow$ `Stack Head` `50`  
g. Push(Residual Pop()) `50`

In [16]:
serial = tl.Serial(
    tl.Select([0, 1, 0, 1]), 
    Addition(), 
    Multiplication(), 
    Addition(), 
    tl.Residual()
)

# Initialization
x = (np.array([3]), np.array([4]))  # input

serial.init(shapes.signature(x))  # initializing serial instance


print("-- Serial Model --")
print(serial, "\n")
print("-- Properties --")
print("name :", serial.name)
print("sublayers :", serial.sublayers)
print("expected inputs :", serial.n_in)
print("promised outputs :", serial.n_out, "\n")

# Inputs
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = serial(x)
print("-- Outputs --")
print("y :", y)

-- Serial Model --
Serial_in2[
  Select[0,1,0,1]_in2_out4
  Addition_in2
  Multiplication_in2
  Addition_in2
  Serial[
    Branch_out2[
      None
      Serial
    ]
    Add_in2
  ]
] 

-- Properties --
name : Serial
sublayers : [Select[0,1,0,1]_in2_out4, Addition_in2, Multiplication_in2, Addition_in2, Serial[
  Branch_out2[
    None
    Serial
  ]
  Add_in2
]]
expected inputs : 2
promised outputs : 1 

-- Inputs --
x : (array([3]), array([4])) 

-- Outputs --
y : [50]


### Trickier Example
Residual layer accept a layer as an argument and it will add the output of that layer to the current stack top input


In [17]:
serial = tl.Serial(
    tl.Select([0, 1, 0, 1]), Addition(), Multiplication(), tl.Residual(Addition()) 
)

# Initialization
x = (np.array([3]), np.array([4]))  # input

serial.init(shapes.signature(x))  # initializing serial instance


print("-- Serial Model --")
print(serial, "\n")
print("-- Properties --")
print("name :", serial.name)
print("sublayers :", serial.sublayers)
print("expected inputs :", serial.n_in)
print("promised outputs :", serial.n_out, "\n")

# Inputs
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = serial(x)
print("-- Outputs --")
print("y :", y)

-- Serial Model --
Serial_in2[
  Select[0,1,0,1]_in2_out4
  Addition_in2
  Multiplication_in2
  Serial_in2[
    Branch_in2_out2[
      None
      Addition_in2
    ]
    Add_in2
  ]
] 

-- Properties --
name : Serial
sublayers : [Select[0,1,0,1]_in2_out4, Addition_in2, Multiplication_in2, Serial_in2[
  Branch_in2_out2[
    None
    Addition_in2
  ]
  Add_in2
]]
expected inputs : 2
promised outputs : 1 

-- Inputs --
x : (array([3]), array([4])) 

-- Outputs --
y : [46]
